In [21]:
from json import loads
from pandas import DataFrame, merge, read_excel
from selenium.webdriver import Chrome, ChromeOptions
from selenium.webdriver.common.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
from selenium.common.exceptions import ElementNotInteractableException, NoSuchElementException, StaleElementReferenceException, TimeoutException
from re import findall
from time import sleep

In [22]:
class ScraperFalabellaCategory:
    def __init__(self):
        chrome_options = ChromeOptions()
        prefs = {"profile.default_content_setting_values.notifications": 2}
        chrome_options.add_experimental_option("prefs", prefs)
        self._driver = Chrome(
            ChromeDriverManager().install(),
            options=chrome_options
        )
        self._wait = WebDriverWait(self._driver, 8)

    def enter_website(self, url):
        self._driver.get(url)
        self._driver.maximize_window()

    def get_element(self, selector, path):
        return self._wait.until(lambda x: x.find_element(selector, path))
    
    def get_elements(self, selector, path):
        return self._wait.until(lambda x: x.find_elements(selector, path))
        
    def close_popups(self):
        self.get_element(By.ID, "testId-accept-cookies-btn").click()
        #self.get_element(By.CLASS_NAME, "dy-lb-close").click()
        
    def extract_products(self, links):
        product_info={
            "Link": [],
            "Nombre": [],
            "Vendedor": [],
            "Precio Regular": [],
            "Precio Descuento": [],
            "Precio Descuento Tarjeta": [],
            "Marca": [],
            "Shipping Details": []
        }
        self._driver.get(links[0] + "?facetSelected=true&f.derived.variant.sellerId=FALABELLA")
        self.get_element(By.XPATH, "//button[@class='Zone-module_modal-close__32xk3']").click()
        for link in links:
            self._driver.get(link + "?facetSelected=true&f.derived.variant.sellerId=FALABELLA")
            visible = True
            while visible:
                products = self.get_elements(By.XPATH, "//div[@class='jsx-1327784995 jsx-97019337 pod pod-4_GRID']")
                for product in products:
                    main_info = product.find_elements(By.XPATH, ".//b[@class]")
                    product_info["Link"].append(link)
                    product_info["Marca"].append(main_info[0].text)
                    product_info["Nombre"].append(main_info[1].text)
                    product_info["Vendedor"].append(main_info[2].text)
                    try:
                        price_cmr = product.find_element(By.XPATH, ".//span[@class='copy10 primary high jsx-2889528833 normal      line-height-22']").text
                    except (StaleElementReferenceException, NoSuchElementException) as error:
                        price_cmr = None

                    try:
                        normal_price = product.find_element(By.XPATH, ".//span[@class='copy3 septenary medium jsx-2889528833 normal    crossed  line-height-17']").text
                    except (StaleElementReferenceException, NoSuchElementException) as error:
                        normal_price = None

                    try:
                        price_internet = product.find_element(By.XPATH, ".//span[@class='copy10 primary medium jsx-2889528833 normal      line-height-22']").text
                    except (StaleElementReferenceException, NoSuchElementException) as error:
                        price_internet = None

                    if not normal_price:
                        normal_price = price_internet
                        price_internet = None

                    product_info["Precio Descuento"].append(price_internet)
                    product_info["Precio Descuento Tarjeta"].append(price_cmr)
                    product_info["Precio Regular"].append(normal_price)

                    shipping_details = product.find_elements(By.XPATH, ".//span[@class='jsx-2122847726 copy4 primary  jsx-2889528833 normal       shipping-text ']")
                    shipping_list = [shipping_details[0].text]
                    try:
                        shipping_list.append(shipping_details[1].text)
                    except IndexError as error:
                        pass
                    product_info["Shipping Details"].append(shipping_list)
                    current_link = self._driver.execute_script("return document.URL")
                try:
                    self._driver.find_element(By.XPATH, "//button[@id='testId-pagination-top-arrow-right']").click()
                    self._wait.until(EC.url_changes(current_link))
                except NoSuchElementException as error:
                    visible = False
                        
        return DataFrame(product_info)

    def extract_products_v2(self, links):
        product_info={
            "Link": [],
            "Nombre": [],
            "Vendedor": [],
            "Precio Regular": [],
            "Precio Descuento": [],
            "Precio Descuento Tarjeta": [],
            "Marca": [],
            "Despacho": [],
            "Retiro": []
        }
        
        
        internet_price=None
        normal_price=None
        cmr_price=None
        for price in data_json["props"]["pageProps"]["results"][0]["prices"]:
            price_type = price["type"]
            if price_type == "internetPrice":
                internet_price = price["price"][0]
            elif price_type == "normalPrice":
                normal_price = price["price"][0]
            else:
                cmr_price = price["price"][0]
                    

In [23]:
scraper = ScraperFalabellaCategory()
scraper.enter_website("https://tienda.falabella.com.pe/falabella-pe")
scraper.close_popups()
df_0 = scraper.extract_products(["https://tienda.falabella.com.pe/falabella-pe/category/cat7350588/Polos-y-Blusas-Niñas"])

In [25]:
df_0.to_csv("Product_sample.csv", index=False, encoding="utf-8-sig")

In [48]:
products = scraper.get_elements(By.XPATH, "//div[@class='jsx-1327784995 jsx-97019337 pod pod-4_GRID']")
product = products[9]

In [76]:
main_info = product.find_elements(By.XPATH, ".//b")
price_cmr = product.find_element(By.XPATH, ".//span[@class='copy10 primary high jsx-2889528833 normal      line-height-22']")
price_internet = product.find_element(By.XPATH, ".//span[@class='copy10 primary medium jsx-2889528833 normal      line-height-22']")
normal_price = product.find_element(By.XPATH, ".//span[@class='copy3 septenary medium jsx-2889528833 normal    crossed  line-height-17']")
flags = product.find_elements(By.XPATH, ".//span[@class='jsx-2122847726 copy4 primary  jsx-2889528833 normal       shipping-text ']")

In [73]:
print(main_info[0].text)
print(main_info[1].text)
print(main_info[2].text)
print(price_cmr.text)
print(price_internet.text)
print(normal_price.text)

INDURAMA
CONGELADOR CI-320GR
Por Falabella
S/ 1,399
S/ 1,499
S/ 1,899


In [79]:
flags[0].text

'Retiro en tienda'

In [3]:
df = read_excel("Falabella_category.xlsx")

In [5]:
df.drop("Unnamed: 0", axis=1, inplace=True)

In [9]:
df_max_level = df[df["Name"] == "Niños y Juguetería"].stack().groupby(level=0).apply(list).tolist()

In [12]:
links = [item[-1] for item in df_max_level]

In [110]:
links[:1]

['https://tienda.falabella.com.pe/falabella-pe/category/cat7350588/Polos-y-Blusas-Niñas']

In [109]:
links

['https://tienda.falabella.com.pe/falabella-pe/category/cat7350588/Polos-y-Blusas-Niñas',
 'https://tienda.falabella.com.pe/falabella-pe/category/cat7350589/Faldas-y-Vestidos-Niñas',
 'https://tienda.falabella.com.pe/falabella-pe/category/cat7350591/Sweaters-y-Polerones-Niñas',
 'https://tienda.falabella.com.pe/falabella-pe/category/cat14620485/Jeans-y-Pantalones-Niñas',
 'https://tienda.falabella.com.pe/falabella-pe/category/cat7350593/Ropa-Interior-y-Pijamas-Niñas',
 'https://tienda.falabella.com.pe/falabella-pe/category/cat7350590/Shorts-Niñas',
 'https://tienda.falabella.com.pe/falabella-pe/category/cat7350592/Casacas-y-abrigos-Niñas',
 'https://tienda.falabella.com.pe/falabella-pe/category/cat16640469/Leggins',
 'https://tienda.falabella.com.pe/falabella-pe/category/cat14620479/Buzos-y-Conjuntos-Niñas',
 'https://tienda.falabella.com.pe/falabella-pe/category/cat11180470/Accesorios',
 'https://tienda.falabella.com.pe/falabella-pe/category/CATG15488/Jardineras-Niñas',
 'https://tien

In [80]:
data = scraper.get_element(By.XPATH, "//script[@id='__NEXT_DATA__']")

In [82]:
data_json = loads(data.get_attribute("text"))

In [88]:
data_json["props"]["pageProps"].keys()

dict_keys(['results', 'layout', 'facets', 'baseFacetsURL', 'appliedFacets', 'searchTerm', 'pagination', 'sortOptions', 'autoCorrections', 'availabilityFacets', 'currentLocationId', 'site', 'categoryParentDetails', 'breadCrumbData', 'metadata', 'categoryBannerHtml', 'canonicalUrl', 'currentUrl', 'pageData', 'isCatSodimacFaviconExp'])

In [92]:
data_json["props"]["pageProps"]["results"][0]
    

{'productId': '881976441',
 'skuId': '881976441',
 'topSpecifications': ['Capacidad total útil: 443 lt',
  'Eficiencia energética: B',
  'Dispensador de agua: No',
  'No Frost: Sí',
  'Ice maker interior: Sí'],
 'merchantCategoryId': 'J11060103',
 'displayName': 'Refrigeradora Bottom Freezer 443 L WRE57AKBPE',
 'productType': 'REGULAR',
 'viewTemplate': 'NORMAL',
 'url': 'https://www.falabella.com.pe/falabella-pe/product/881976441/Refrigeradora-Bottom-Freezer-443-L-WRE57AKBPE',
 'brand': 'WHIRLPOOL',
 'media': {'id': '881976441',
  'type': 'PRODUCT',
  'onImageHover': 'FETCH_SECOND_IMAGE'},
 'mediaUrls': ['https://falabella.scene7.com/is/image/FalabellaPE/881976441_1',
  'https://falabella.scene7.com/is/image/FalabellaPE/881976441_2',
  'https://falabella.scene7.com/is/image/FalabellaPE/881976441_3',
  'https://falabella.scene7.com/is/image/FalabellaPE/881976441_4',
  'https://falabella.scene7.com/is/image/FalabellaPE/881976441_5',
  'https://falabella.scene7.com/is/image/FalabellaPE/8

In [95]:
print(data_json["props"]["pageProps"]["results"][0]["productId"])
print(data_json["props"]["pageProps"]["results"][0]["displayName"])
print(data_json["props"]["pageProps"]["results"][0]["url"])
print(data_json["props"]["pageProps"]["results"][0]["brand"])

for price in data_json["props"]["pageProps"]["results"][0]["prices"]:
    price_type = price["type"]
    if price_type == "internetPrice":
        internet_price = price["price"][0]
    elif price_type == "normalPrice":
        normal_price = price["price"][0]
    else:
        cmr_price = price["price"][0]
print(data_json["props"]["pageProps"]["results"][0]["availability"]["homeDeliveryShipping"])
print(data_json["props"]["pageProps"]["results"][0]["availability"]["pickUpFromStoreShipping"])
print(data_json["props"]["pageProps"]["results"][0]["sellerName"])

881976441
Refrigeradora Bottom Freezer 443 L WRE57AKBPE
https://www.falabella.com.pe/falabella-pe/product/881976441/Refrigeradora-Bottom-Freezer-443-L-WRE57AKBPE
WHIRLPOOL
[{'label': '', 'icons': '', 'symbol': 'S/ ', 'type': 'internetPrice', 'crossed': False, 'price': ['2,999']}, {'label': '', 'icons': '', 'symbol': 'S/ ', 'type': 'normalPrice', 'crossed': True, 'price': ['4,289']}]
Despacho a Domicilio
Retiro en tienda
FALABELLA


In [97]:
%%timeit
internet_price=None
normal_price=None
cmr_price=None
for price in data_json["props"]["pageProps"]["results"][0]["prices"]:
    price_type = price["type"]
    if price_type == "internetPrice":
        internet_price = price["price"][0]
    elif price_type == "normalPrice":
        normal_price = price["price"][0]
    else:
        cmr_price = price["price"][0]

755 ns ± 19.9 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [98]:
print(data_json["props"]["pageProps"]["results"][0]["availability"])

{'homeDeliveryShipping': 'Despacho a Domicilio', 'pickUpFromStoreShipping': 'Retiro en tienda', 'internationalShipping': '', 'primeShipping': ''}
